In [1]:
import os
import pandas as pd
import pyomo.environ as pyo

## Read the data from csv using pandas

In [2]:
csv_file_path = "./delivery_costs.csv"

In [3]:
df = pd.read_csv(csv_file_path, index_col=0)

In [4]:
df

,NYC,LA,Chicago,Houston
Harlingen,1956,1606,1410,330
Memphis,1906,1792,531,567
Ashland,485,2322,324,1236


In [5]:
N = list(df.index.map(str))
M = list(df.columns.map(str))
d = {(r, c):df.loc[r,c] for r in N for c in M}
P = 2

## Create the mode

In [6]:
model = pyo.ConcreteModel(name = "(WL)")

model.x = pyo.Var(N, M, bounds=(0, 1))
model.y = pyo.Var(N, within=pyo.Binary)

def obj_rule(model):
    return sum(d[n,m]*model.x[n,m] for n in N for m in M)

model.obj = pyo.Objective(rule=obj_rule)

def one_per_cust_rule(model, m):
    return sum(model.x[n,m] for n in N) == 1

model.one_per_cust = pyo.Constraint(M, rule=one_per_cust_rule)

def warehouse_active_rule(model, n, m):
    return model.x[n,m] <= model.y[n]

model.warehouse_active = pyo.Constraint(N, M, rule=warehouse_active_rule)

def num_warehouses_rule(model):
    return sum(model.y[n] for n in N) <= P
model.num_warehouses = pyo.Constraint(rule=num_warehouses_rule)


## Solve

In [7]:
solver = pyo.SolverFactory('glpk')
res = solver.solve(model)

In [8]:
# model.pprint()

In [9]:
# produce nicely formatted output
for wl in N:
    if pyo.value(model.y[wl]) > 0.5:
        customers = [str(cl) for cl in M if pyo.value(model.x[wl, cl] > 0.5)]
        print(f"{wl} + serves customers: {customers}")
    else:
        print(str(wl)+": do not build")


Harlingen + serves customers: ['LA', 'Houston']
Memphis: do not build
Ashland + serves customers: ['NYC', 'Chicago']
